# Задача

Изучить данные.

# Импорты

In [1]:
from collections import Counter
import re

import pandas as pd

# Загрузка данных

In [2]:
data_path = '../data/train.csv'

df = pd.read_csv(data_path)

# Изучение

In [3]:
pd.set_option('display.max_colwidth', 100)

df.sample(10)

,text,label
8373,"если вот так коротко то что-то понятно, но вот читая доку я начинаю тупить",0
4973,"можно побенчмаркать, но я не думаю что будет принципиальная разница",0
2609,"»Возможность быть причастным к крупным интересным проектам, которые не стыдно показать в портфол...",0
3214,у нас три раза сажают только одного и того же человека на один и тот же пост,1
11237,я не с этой эпохи... что есть АКБ? аккумуляторная батарея,1
9443,в работе разнорабочим у белых красивых и добрых людей за океаном,1
6029,сейчас бы нарушить NDA и назвать з/п в чате на 684 человека,1
11639,Ок) просто недавно подобную вещь делал - остались наработки.,1
3190,"чтобы деплоящие макакены не запутались в разнообразии, но ладно, объясню так",0
3562,"понял, очень внятно объяснили, ""light-weight threading"" как раз из gevent в голове подтянулось, ...",0


In [4]:
len(df)

12404

### Распределение классов

In [5]:
df['label'].value_counts(normalize=True)

1    0.555063
0    0.444937
Name: label, dtype: float64

Почти поровну.

## Тексты

In [6]:
df['text_length'] = df['text'].apply(lambda x: len(x))

In [7]:
df['text_length'].describe()

count    12404.000000
mean       100.786359
std         78.801220
min         25.000000
25%         61.000000
50%         79.000000
75%        114.000000
max       2434.000000
Name: text_length, dtype: float64

In [8]:
df.groupby(by='label')['text_length'].describe()

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
0,5519.0,109.537779,92.054401,30.0,63.0,84.0,124.0,2434.0
1,6885.0,93.771242,65.458823,25.0,59.0,75.0,107.0,1184.0


Тексты по первому классу чуть короче.

#### Посмотрим на уникальные символы

In [9]:
zero_class_symb_counter = Counter()
first_class_symb_counter = Counter()

for row in df.iterrows():
    if row[1]['label'] == 0:
        for s in row[1]['text']:
            zero_class_symb_counter[s] += 1
    else:
        for s in row[1]['text']:
            first_class_symb_counter[s] += 1

In [10]:
len(zero_class_symb_counter)

225

In [11]:
zero_class_symb_counter.most_common(50)

[(' ', 94135),
 ('о', 46423),
 ('е', 35339),
 ('т', 33266),
 ('а', 33249),
 ('н', 26669),
 ('и', 26637),
 ('с', 19771),
 ('р', 18574),
 ('л', 15461),
 ('в', 15353),
 ('к', 14748),
 ('п', 13086),
 ('м', 12638),
 ('у', 12019),
 ('д', 11994),
 ('ь', 9093),
 ('б', 7591),
 ('я', 7579),
 ('ы', 7556),
 (',', 7376),
 ('ч', 7012),
 ('з', 6703),
 ('.', 6214),
 ('n', 6102),
 ('e', 5750),
 ('г', 5342),
 ('t', 4566),
 ('й', 4279),
 ('ж', 4234),
 ('i', 3632),
 ('s', 3551),
 ('r', 3452),
 ('a', 3441),
 ('o', 3366),
 ('\\', 3167),
 ('х', 2933),
 ('ш', 2914),
 ('ю', 2661),
 ('l', 2323),
 ('p', 2135),
 ('-', 2001),
 ('э', 1882),
 ('c', 1846),
 ('d', 1846),
 (')', 1832),
 ('m', 1551),
 ('u', 1464),
 ('ф', 1385),
 ('ц', 1384)]

In [12]:
len(first_class_symb_counter)

225

In [13]:
first_class_symb_counter.most_common(50)

[(' ', 101588),
 ('о', 53784),
 ('е', 42464),
 ('а', 40999),
 ('т', 38293),
 ('и', 31704),
 ('н', 31374),
 ('с', 23401),
 ('р', 21080),
 ('л', 19625),
 ('в', 17911),
 ('к', 17520),
 ('м', 15709),
 ('д', 14888),
 ('п', 14152),
 ('у', 14093),
 ('ь', 10101),
 (',', 9633),
 ('б', 9150),
 ('я', 9062),
 ('ч', 8884),
 ('ы', 8773),
 ('з', 7296),
 ('г', 7012),
 ('.', 5605),
 ('ж', 4873),
 ('й', 4851),
 ('х', 3945),
 ('ш', 3719),
 ('ю', 3058),
 ('-', 2537),
 ('э', 2234),
 (')', 1933),
 ('n', 1923),
 ('?', 1836),
 ('0', 1703),
 ('ф', 1585),
 ('ц', 1556),
 ('щ', 1472),
 ('e', 1375),
 ('t', 1182),
 ('a', 1171),
 ('o', 1111),
 ('1', 1093),
 ('Н', 1090),
 ('\\', 1075),
 ('i', 999),
 ('s', 981),
 ('r', 938),
 ('"', 742)]

## Слова

In [14]:
def split_words(text):
    return re.split('[\W\d_]+', text)

In [15]:
split_words(df.sample(1).iloc[0]['text'])

['как',
 'одно',
 'ты',
 'сказал',
 'оно',
 'перезапускается',
 'А',
 'если',
 'оно',
 'перезапускается',
 'запуститься',
 'может',
 'новая',
 'версия']

In [16]:
df['words_count'] = df['text'].apply(lambda x: len(split_words(x)))

In [17]:
df.groupby(by='label')['words_count'].describe()

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
0,5519.0,18.340823,13.618048,6.0,11.0,14.0,21.0,365.0
1,6885.0,16.062600,10.048980,2.0,11.0,13.0,18.0,192.0


И "слов" в первом классе в среднем чуть меньше.

#### Посмотрим на уникальные слова

In [18]:
zero_class_word_counter = Counter()
first_class_word_counter = Counter()

for row in df.iterrows():
    if row[1]['label'] == 0:
        for w in split_words(row[1]['text']):
            zero_class_word_counter[w] += 1
    else:
        for w in split_words(row[1]['text']):
            first_class_word_counter[w] += 1

In [19]:
len(zero_class_word_counter)

20159

In [20]:
zero_class_word_counter.most_common(20)

[('', 2694),
 ('не', 2481),
 ('в', 2408),
 ('и', 2213),
 ('что', 1482),
 ('n', 1402),
 ('на', 1340),
 ('то', 1047),
 ('это', 951),
 ('с', 931),
 ('я', 919),
 ('а', 796),
 ('как', 776),
 ('по', 597),
 ('ты', 594),
 ('у', 556),
 ('если', 544),
 ('но', 525),
 ('так', 523),
 ('для', 454)]

In [21]:
zero_class_word_counter.most_common()[1000:1020]

[('вывод', 12),
 ('деньги', 12),
 ('готов', 12),
 ('системы', 12),
 ('имел', 12),
 ('пхп', 12),
 ('вызывать', 12),
 ('ними', 12),
 ('Без', 12),
 ('попробовать', 12),
 ('default', 12),
 ('that', 12),
 ('g', 12),
 ('опыт', 12),
 ('пофиг', 12),
 ('Нужно', 12),
 ('образом', 12),
 ('параметр', 12),
 ('запуска', 12),
 ('посмотри', 12)]

In [22]:
zero_class_word_counter.most_common()[::-1][:20]

[('PWA', 1),
 ('наработках', 1),
 ('версткой', 1),
 ('подкупает', 1),
 ('спорят', 1),
 ('вкусах', 1),
 ('предпочтения', 1),
 ('вкусы', 1),
 ('зачётно', 1),
 ('фрагмент', 1),
 ('разделе', 1),
 ('маркете', 1),
 ('популярно', 1),
 ('школьником', 1),
 ('инвестиция', 1),
 ('маржи', 1),
 ('xmpp', 1),
 ('привата', 1),
 ('ифапдаун', 1),
 ('нетфорк', 1)]

In [23]:
len(first_class_word_counter)

23416

In [24]:
first_class_word_counter.most_common(20)

[('', 3374),
 ('не', 2927),
 ('и', 2783),
 ('в', 2721),
 ('что', 1927),
 ('на', 1859),
 ('то', 1318),
 ('я', 1274),
 ('это', 1171),
 ('а', 1170),
 ('с', 1032),
 ('как', 935),
 ('по', 799),
 ('у', 771),
 ('но', 705),
 ('так', 696),
 ('если', 630),
 ('же', 605),
 ('ты', 590),
 ('ну', 585)]

In [25]:
first_class_word_counter.most_common()[1000:1020]

[('GPU', 11),
 ('машин', 11),
 ('настолько', 11),
 ('футболки', 11),
 ('деревьев', 11),
 ('карту', 11),
 ('ошибку', 11),
 ('прирост', 11),
 ('ноль', 11),
 ('мире', 11),
 ('mlbootcamp', 11),
 ('оценки', 11),
 ('Еще', 11),
 ('иметь', 10),
 ('попробовал', 10),
 ('оставить', 10),
 ('сообщение', 10),
 ('простой', 10),
 ('заниматься', 10),
 ('количества', 10)]

In [26]:
first_class_word_counter.most_common()[::-1][:20]

[('квору', 1),
 ('Посмотри', 1),
 ('НЛП', 1),
 ('сигмоидой', 1),
 ('Архитектура', 1),
 ('неправыми', 1),
 ('докормить', 1),
 ('далёкую', 1),
 ('кормить', 1),
 ('вычислят', 1),
 ('колода', 1),
 ('чья', 1),
 ('Манчкине', 1),
 ('жизнеспособен', 1),
 ('leave', 1),
 ('тормозах', 1),
 ('поглотила', 1),
 ('свистом', 1),
 ('улетит', 1),
 ('передастся', 1)]

## Выводы

Язык жаргонный, слова часто коверкаются. Можно предложить либо всякое приведение, лемминг-стемминг. Либо энграмы. либо и то и другое.

Можно попробовать обучить фасттекст, а затем бустинг или RNN. Вряд ли предобученные эмбединги тут будут хороши. Но На это всё вряд ли хватит времени, поэтому наш ответ -- катбуст. Только нужно будет перед ним пройтись приведением слов к базовой форме.